# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: GPT-2
* Evaluation approach: Evaluate method with Hugging Face Trainer
* Fine-tuning dataset: polsci/fake-news

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
!pip -q install -U transformers datasets accelerate peft

In [7]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

from datasets import load_dataset

from peft import (
    LoraConfig,
    get_peft_model,
    PeftModel,
    PeftConfig
)

import numpy as np
from sklearn.metrics import accuracy_score, f1_score


In [9]:
dataset = load_dataset("polsci/fake-news")

TEXT_COL_CANDIDATES = ["text", "content", "article", "title"]
LABEL_COL_CANDIDATES = ["label", "labels", "target", "y"]

def pick_col(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    raise ValueError(f"Uygun kolon bulunamadı. Mevcut kolonlar: {cols}")

cols = set(dataset["train"].column_names)
text_col = pick_col(cols, TEXT_COL_CANDIDATES)
label_col = pick_col(cols, LABEL_COL_CANDIDATES)

if set(dataset.keys()) == {"train"}:
    ds_split = dataset["train"].train_test_split(
        test_size=0.2,
        seed=42,
        stratify_by_column=label_col if label_col in dataset["train"].column_names else None
    )
    train_ds = ds_split["train"]
    eval_ds  = ds_split["test"]
else:
    train_ds = dataset["train"] if "train" in dataset else None
    eval_split_name = "test" if "test" in dataset else "validation"
    eval_ds = dataset[eval_split_name]

def take_subset(ds, n=1000):
    return ds.select(range(min(n, len(ds))))

train_ds = take_subset(train_ds, 1000)
eval_ds  = take_subset(eval_ds, 1000)


In [10]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

max_len = 256

def preprocess(batch):
    toks = tokenizer(
        batch[text_col],
        padding="max_length",
        truncation=True,
        max_length=max_len
    )
    toks["labels"] = batch[label_col]
    return toks

train_ds_tok = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
eval_ds_tok  = eval_ds.map(preprocess,  batched=True, remove_columns=eval_ds.column_names)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [11]:
num_labels = int(max(max(train_ds[label_col]), max(eval_ds[label_col])) + 1)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="single_label_classification"
)

model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }


In [16]:
args = TrainingArguments(
    output_dir="./tmp-gpt2-eval",
    per_device_eval_batch_size=16,
    do_train=False,
    do_eval=True,
    logging_strategy="no",
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    eval_dataset=eval_ds_tok,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate()
print(metrics)

/tmp/ipython-input-2008211549.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 2.9735851287841797, 'eval_model_preparation_time': 0.0054, 'eval_accuracy': 0.4479166666666667, 'eval_f1_macro': 0.32475116124751163, 'eval_runtime': 149.9594, 'eval_samples_per_second': 0.64, 'eval_steps_per_second': 0.04}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [17]:
import os
import torch
from peft import LoraConfig, get_peft_model

os.environ["WANDB_DISABLED"] = "true"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"

if getattr(model.config, "pad_token_id", None) is None and tokenizer.pad_token_id is not None:
    model.config.pad_token_id = tokenizer.pad_token_id


In [18]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=["c_attn", "c_proj"]
)

In [19]:
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

from transformers import TrainingArguments, Trainer

trainable params: 812,544 || all params: 125,253,888 || trainable%: 0.6487


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:2156: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [27]:
train_args = TrainingArguments(
    output_dir="./runs-gpt2-fakenews-lora",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.1,
    gradient_accumulation_steps=1,
    logging_steps=50,
    save_strategy="no",
    report_to=[],
    fp16=torch.cuda.is_available(),
    do_eval=True
)

trainer = Trainer(
    model=peft_model,
    args=train_args,
    processing_class=tokenizer,
    train_dataset=train_ds_tok,
    eval_dataset=eval_ds_tok,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()
eval_metrics_ft = trainer.evaluate()
print("Fine-tuned eval metrics:", eval_metrics_ft)

save_dir = "./gpt2-fake-news-lora"
peft_model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"LoRA adapter weights saved to: {save_dir}")

Step,Training Loss


Fine-tuned eval metrics: {'eval_loss': 0.6620649695396423, 'eval_accuracy': 0.6145833333333334, 'eval_f1_macro': 0.6094557449147884, 'eval_runtime': 96.3039, 'eval_samples_per_second': 0.997, 'eval_steps_per_second': 0.062, 'epoch': 1.0}
LoRA adapter weights saved to: ./gpt2-fake-news-lora


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [ ]:
# Saving the model
# model.save("/tmp/your_model_name")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [34]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer

In [35]:
peft_model_id = "./gpt2-fake-news-lora"

config = PeftConfig.from_pretrained(peft_model_id)

In [36]:
ft_tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
if ft_tokenizer.pad_token is None:
    ft_tokenizer.pad_token = ft_tokenizer.eos_token
ft_tokenizer.padding_side = "right"

In [37]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path,
    num_labels=model.config.num_labels
)
base_model.config.pad_token_id = ft_tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
ft_model = PeftModel.from_pretrained(base_model, peft_model_id)

try:
    ft_model.get_base_model().config.pad_token_id = ft_tokenizer.pad_token_id
except Exception:
    pass


ft_trainer = Trainer(
    model=ft_model,
    args=train_args,
    processing_class=ft_tokenizer,
    eval_dataset=eval_ds_tok,
    compute_metrics=compute_metrics
)

eval_metrics_reload = ft_trainer.evaluate()
print("Reloaded PEFT model eval metrics:", eval_metrics_reload)

Reloaded PEFT model eval metrics: {'eval_loss': 0.6620649695396423, 'eval_model_preparation_time': 0.0061, 'eval_accuracy': 0.6145833333333334, 'eval_f1_macro': 0.6094557449147884, 'eval_runtime': 93.9358, 'eval_samples_per_second': 1.022, 'eval_steps_per_second': 0.064}


In [39]:

label_names = dataset["train"].features[label_col].names
print("Label names:", label_names)

id2label = {i: name for i, name in enumerate(label_names)}
label2id = {name: i for i, name in enumerate(label_names)}

ft_model.config.id2label = id2label
ft_model.config.label2id = label2id

text = "Breaking: Government announces a new economic package today."
inputs = ft_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)

with torch.no_grad():
    outputs = ft_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1).squeeze()

pred_id = int(torch.argmax(probs))
print("Predicted label:", ft_model.config.id2label[pred_id])
print("Probabilities:", {ft_model.config.id2label[i]: float(p) for i, p in enumerate(probs)})


Label names: ['legit', 'fake']
Predicted label: legit
Probabilities: {'legit': 0.7979366779327393, 'fake': 0.20206332206726074}


In [40]:
def classify_text(text: str):
    inputs = ft_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = ft_model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1).squeeze()

    pred_id = int(torch.argmax(probs))
    pred_label = ft_model.config.id2label[pred_id]
    return pred_label, {ft_model.config.id2label[i]: float(p) for i, p in enumerate(probs)}

samples = [
    "Government announces new tax reform today.",
    "Aliens landed in New York and took over Times Square!"
]

for s in samples:
    label, probs = classify_text(s)
    print(f"\nText: {s}\nPredicted: {label}\nProbabilities: {probs}")



Text: Government announces new tax reform today.
Predicted: legit
Probabilities: {'legit': 0.8178446292877197, 'fake': 0.1821553260087967}

Text: Aliens landed in New York and took over Times Square!
Predicted: legit
Probabilities: {'legit': 0.7640143632888794, 'fake': 0.2359856367111206}
